In [ ]:
from src import *  
from src.extra import ExEstimates  

from src.parallelize import parallelize  # Simple multiprocessing implementation using dask

# Basic

In [ ]:
# Use LoadEstimates to load a saved Estimates instance from a file
# The loaded Estimates instance contains various data stored as attributes
# When printed, it shows the stored data
c = LoadEstimates('data/ideal/di_3px_f0.14_d0.npz')


# The data in this repository is already processed
# You can use the est() method to perform estimation, the method is defined in src/estimator.py
c.est() # Perform estimation using the loaded data

plt.plot(c.time_domain[0])
plt.xlabel(r'$n$')
plt.ylabel(r'$\hat s$')
plt.show()

plt.scatter(np.arange(200), c.estimates)
plt.xlabel(r'Group')
plt.ylabel(r'$\hat f$')
plt.axhline(0.14, color='tab:red')
plt.show()

# Simulator

In [ ]:
# Create simulator object
# DMD.A(x) = x * DMD.PIXEL_SIZE / 2, for quick amplitude calculation
# This Simulator represents a DI simulation with frequency 0.34 and amplitude 5/2 DMD Pixels
# Use measurement = 'hg' or 'hg_spade' for HG-SPADE
sim = Simulator(
    measurement = 'DI',
    amplitude = DMD.A(5),
    ground_truth = 0.34,
    waveform = 'sign'
    )


# gen() returns a standard Estimates instance
# gen() accepts different signal and noise photon counts, default DI 400, SPADE 60
c = sim.gen()


# Estimate the generated data (non-extended Estimates uses single-parameter LSE, same as before)
c = c.est()
x = np.arange(200)
plt.plot(x, c.estimates, 'v', label='Single-Params; LSE')


# The non-extended Estimates (using single-parameter LSE) is the method actually used in the paper
# ExEstimates extends this functionality but isn't used in the current setup
c = ExEstimates(c)


# Perform LSE estimation (multi-parameter)
c = c.lse()
plt.plot(x, c.extra_estimates_f, '^', label='Multi-Params; LSE')


# Perform MLE estimation (multi-parameter, Kay1993, slight difference from multi-parameter LSE due to approximation)
c = c.mle()


plt.plot(x, c.extra_estimates_f, 'o', label='Multi-Params; MLE')
plt.xlabel('Group')
plt.ylabel(r'$\hat f$')
plt.legend()
plt.show()

In [ ]:
# MLE amplitude matches the theoretical value
c.extra_estimates_a.mean(), (4/pi) * DMD.A(5)

In [ ]:
# Experimental frequency range from 0.1 to 0.4
freq_list = np.linspace(0.1, 0.4, 61)

# The two parameters in this tuple will generate random numbers through np.random.normal(*delay_params)
# delay_params is used to simulate delays, with the first value being the mean and the second the standard deviation
# An additional 1e-6 is added for DMD and other instrument delays
delay_params = (2.8e-3 + 1e-6, 0.48e-3)

In [ ]:
# Since the simulation algorithm is not vectorized, it is not efficient, so multiprocessing is used to speed it up
@parallelize() # This decorator converts the first parameter of the original function to accept computation graph input
def run(f, m):
    c = Simulator(m, DMD.A(3), np.round(f, 5), 'sign', delay_params).gen()
    c = c.est()
    c.savez('TEMP/') # TEMP folder is not synced with git, savez() only needs a storage path, program will auto-name it.
    return c

_ = run(freq_list, 'DI'), run(freq_list, 'SPADE')

In [ ]:
for m in ('SPADE', 'DI'):
    f_var = []
    for i, f in enumerate(freq_list):
        meta = MetaData(m, np.round(f, 5), DMD.A(3))
        c = LoadEstimates(f'TEMP/{meta.convert2str()}.npz')
        f_var.append(c.estimates.var(ddof=1) * c.photons)
    plt.plot(freq_list, f_var, 'o' if m == 'SPADE' else '^', label=m)

# FI calculation for SPADE and DI classes
# Since CFI is the same in the absence of noise, either method can be used
fi = DI.ApproxCFI((4/pi) * DMD.A(3))
plt.axhline(1/fi, color='tab:red', zorder=1, label='QCRB')  # Plot QCRB

plt.legend()
plt.xlim(0.1, 0.4)
plt.xlabel(r'$f$')
plt.ylabel(r'$\nu\operatorname{Var}(\hat f)$')
plt.show()
